<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/Chatbottrial7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. IMPORT EVERYTHING
2.   SET JSON FILE AS PANDAS DATAFRAME
3.   PREPROCESS THE DATA
4.   SPLIT DATA INTO TRAINING AND TESTING DATA





In [136]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import SGD
#from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import re
import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#import the dataset
with open('/content/dataset.json') as f:
    intents = json.load(f)

# Preprocess Data-remove stopwords
stop_words = set(stopwords.words('english'))

data = []
labels = []

#perform tokenization, filtering, lemmatizatiopn
for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern = pattern.strip() # Remove leading/trailing whitespace
        tokenized_pattern = nltk.word_tokenize(pattern)
        filtered_pattern = [word.lower() for word in tokenized_pattern if word.isalnum() and not word in stop_words]
        lemmatizer = WordNetLemmatizer()
        lemmatized_pattern = lemmatized_pattern = [lemmatizer.lemmatize(word) for word in filtered_pattern]
        data.append(' '.join(lemmatized_pattern))
        labels.append(intent['tag'])
#The data list contains all the patterns, and the labels list contains all the corresponding intent tags.
# Create a Pandas DataFrame
df = pd.DataFrame({'Patterns': data, 'Tags': labels})

# Encode the Labels by converting tags into numerical form
encoder = LabelEncoder()
df['Tags'] = encoder.fit_transform(df['Tags'])
df['Tags'] = to_categorical(df['Tags'])

The encoding of the labels: The output of this code will be a DataFrame where the 'Tags' column has been converted into one-hot encoded binary values. This is useful for feeding the data into a neural network for text classification, where each class of text can be represented as a one-hot encoded vector.




In [ ]:
df

In [32]:
import collections
word_counts = collections.Counter(data)
num_words = len(word_counts)

print(f"The number of unique words in the vocabulary is {num_words}")

The number of unique words in the vocabulary is 425


PERFORM FEATURE EXTRACTION USING TDIF

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
#The CountVectorizer converts the text data into numerical vectors by counting the occurrence of each word in the text
vectorizer = CountVectorizer()
vectorizer.fit(data)

CountVectorizer()

In [46]:
X = vectorizer.transform(data) # transform text to feature vectors

In [71]:
# Encode labels as unique intergers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
y_train = np.array(labels)


In [83]:
print(X_train.shape[0])
print(len(y_train))

361
361


In [84]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train)

In [85]:
print(X_train.shape[0])
print(len(y_train))

361
361


In [ ]:
print(X_test)

In [ ]:
print(y_train)

In [ ]:
print(y_test)

In [96]:
num_classes = len(le.classes_)

In [124]:
model = Sequential()
model.add(Embedding(len(vectorizer.vocabulary_), 100, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [125]:
print(y_train.shape)
print(model.predict(X_train).shape)

(361,)
12/12 [==============================] - 2s 120ms/step
(361, 254)


In [87]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 510, 100)          51000     
                                                                 
 bidirectional_4 (Bidirecti  (None, 510, 128)          84480     
 onal)                                                           
                                                                 
 dropout_8 (Dropout)         (None, 510, 128)          0         
                                                                 
 dense_8 (Dense)             (None, 510, 64)           8256      
                                                                 
 dropout_9 (Dropout)         (None, 510, 64)           0         
                                                                 
 dense_9 (Dense)             (None, 510, 254)          16510     
                                                      

In [126]:
y_train = y_train.reshape(-1, 1)

In [127]:
num_classes = len(np.unique(y_train))

In [128]:
y_train = np.resize(y_train, (num_classes,))

In [137]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = SGD(learning_rate=0.01,
                            momentum=0.9, #help to improve the training speed and model stability
                            nesterov=True), metrics=['accuracy'])

In [138]:
num_classes = model.layers[-1].units

In [139]:
y_train = np.resize(y_train, (X_train.shape[0],))

In [140]:
print(X_train.shape)
print(y_train.shape)

(361, 510)
(361,)


In [141]:
model.fit(X_train, y_train, epochs=20,  batch_size=5, verbose=1)

Epoch 1/20
73/73 [==============================] - 34s 413ms/step - loss: 4.5262 - accuracy: 0.0416
Epoch 2/20
73/73 [==============================] - 31s 422ms/step - loss: 4.5094 - accuracy: 0.0499
Epoch 3/20
73/73 [==============================] - 30s 412ms/step - loss: 4.4967 - accuracy: 0.0416
Epoch 4/20
73/73 [==============================] - 30s 410ms/step - loss: 4.5172 - accuracy: 0.0499
Epoch 5/20
73/73 [==============================] - 31s 423ms/step - loss: 4.4828 - accuracy: 0.0416
Epoch 6/20
73/73 [==============================] - 30s 408ms/step - loss: 4.4928 - accuracy: 0.0499
Epoch 7/20
73/73 [==============================] - 30s 411ms/step - loss: 4.4988 - accuracy: 0.0332
Epoch 8/20
73/73 [==============================] - 30s 411ms/step - loss: 4.4431 - accuracy: 0.0471
Epoch 9/20
73/73 [==============================] - 30s 409ms/step - loss: 4.4362 - accuracy: 0.0416
Epoch 10/20
73/73 [==============================] - 30s 410ms/step - loss: 4.4485 - accura